# Hipotezių testavimas su Python (basic statistics)

Baziniai hipotezių testavimo pavyzdžiai, dažniausiai naudojami duomenų analitikoje ir verslo kontekste.
Dėmesys skiriamas praktinei logikai: hipotezių formulavimui, testo pasirinkimui, prielaidoms, rezultatų interpretacijai ir tipinėms klaidoms.


## Naudojamos bibliotekos

- pandas: duomenų paruošimas ir grupavimas
- numpy: skaitiniai skaičiavimai
- scipy.stats: statistiniai testai (t testai, chi kvadrato testas, koreliacija)
- statsmodels: pagalbinės funkcijos, pvz., proporcijų testai (pasirinktinai)

Pastaba: realiame darbe daugiausia laiko skiriama duomenų kokybei ir paruošimui. Testas yra tik paskutinis žingsnis.


In [ ]:
import numpy as np
import pandas as pd

from scipy import stats


## Pagrindinės sąvokos (trumpai)

- H0 (nulinė hipotezė): nėra efekto / skirtumo.
- Ha (alternatyvi hipotezė): yra efektas / skirtumas.
- α (reikšmingumo lygis): dažniausiai 0.05. Tai klaidos riba, kuri pasirenkama iš anksto.
- p reikšmė: tikimybė gauti tokį arba labiau ekstremalų rezultatą, jei H0 būtų teisinga.

Sprendimo taisyklė:
- jei p ≤ α, H0 atmetama;
- jei p > α, nėra pakankamai įrodymų atmesti H0.


## Duomenų generavimas demonstracijai

Toliau naudojami dirbtinai sugeneruoti duomenys, kad būtų aiškiai matoma testo logika.
Realiose situacijose duomenys atkeliauja iš duomenų bazės, failų, įvykių žurnalų ar analitikos įrankių.


In [3]:
np.random.seed(42)

n = 60

# A/B pavyzdys: vidutinė užsakymo suma (EUR) dviem grupėms
group_A = np.random.normal(loc=42, scale=8, size=n)  # kontrolė
group_B = np.random.normal(loc=45, scale=8, size=n)  # testas (šiek tiek didesnis vidurkis)

df_ab = pd.DataFrame({
    "group": ["A"] * n + ["B"] * n,
    "order_value": np.concatenate([group_A, group_B])
})

df_ab.head()


,group,order_value
0,A,45.973713
1,A,40.893886
2,A,47.181508
3,A,54.184239
4,A,40.126773


## 1) Dviejų nepriklausomų grupių vidurkių palyginimas: t testas

Verslo klausimas:
- Ar A/B testo grupė B turi didesnę vidutinę užsakymo sumą nei grupė A?

Hipotezės:
- H0: grupių vidurkiai vienodi (nėra efekto).
- Ha: grupių vidurkiai skiriasi (yra efektas).

Praktikoje dažnai tikrinama viena kryptis (ar B padidėjo), tačiau pradžioje saugiau rodyti dvipusį testą.


In [4]:
a = df_ab.loc[df_ab["group"] == "A", "order_value"].values
b = df_ab.loc[df_ab["group"] == "B", "order_value"].values

a_mean, b_mean = a.mean(), b.mean()
a_mean, b_mean


(40.76276253543608, 44.97053258366812)

### Prielaidos (minimaliai)

t testas remiasi šiomis prielaidomis:
- duomenys yra skaitiniai;
- stebėjimai grupėse yra nepriklausomi;
- pasiskirstymas yra artimas normaliam (arba imtis pakankamai didelė);
- dispersijos panašios (jei nenaudojama Welch korekcija).

Geroji praktika:
- pagal nutylėjimą naudoti Welch t testą (equal_var=False), nes jis saugesnis, kai dispersijos skiriasi.


In [5]:
t_stat, p_value = stats.ttest_ind(a, b, equal_var=False)  # Welch t testas
t_stat, p_value


(-3.110727229567604, 0.002341270150177661)

### Rezultato interpretacija

- p reikšmė nurodo, ar skirtumas statistiškai reikšmingas.
- verslo sprendimui svarbus ir efekto dydis: vidurkių skirtumas bei jo mastas.

Dažna klaida:
- p reikšmė interpretuojama kaip „tikimybė, kad H0 teisinga“. Tai neteisinga.

Atliktas Welch t testas, kuriuo lyginami dviejų nepriklausomų grupių vidurkiai (neprielaidaujant lygių dispersijų).

t statistika = -3.11  
p reikšmė = 0.00234

Neigiama t reikšmė reiškia, kad pirmos grupės (a) vidurkis yra mažesnis nei antros grupės (b) vidurkis.

p = 0.00234 < 0.05, todėl nulinė hipotezė (kad grupių vidurkiai lygūs) atmetama 5 % reikšmingumo lygyje.

Išvada: tarp grupių vidurkių yra statistiškai reikšmingas skirtumas. Skirtumas mažai tikėtinas atsitiktinumo rezultatas.



In [6]:
diff = b_mean - a_mean
diff


4.207770048232035

### Efekto dydis: Cohen d (paprasta versija)

Cohen d aprašo skirtumo dydį standartinių nuokrypių vienetais.
Tai padeda atskirti situacijas, kai p reikšmė maža, bet praktinis efektas yra nereikšmingas.

Dažna klaida:
- remtis vien p reikšme, neįvertinant efekto dydžio.


In [7]:
def cohens_d(x, y):
    # Cohen d (naudojamas sujungtas SD; Welch atveju tai apytikslis rodiklis)
    nx, ny = len(x), len(y)
    sx, sy = x.std(ddof=1), y.std(ddof=1)
    s_pooled = np.sqrt(((nx - 1) * sx**2 + (ny - 1) * sy**2) / (nx + ny - 2))
    return (y.mean() - x.mean()) / s_pooled

d = cohens_d(a, b)
d


0.5679384912932427

Cohen’s d matuoja skirtumo tarp dviejų grupių vidurkių dydį standartinių nuokrypių vienetais.

0.568 reiškia, kad antros grupės (b) vidurkis yra apie 0.57 standartinio nuokrypio didesnis nei pirmos grupės (a).

Pagal įprastą interpretaciją:
0.2 – mažas efektas  
0.5 – vidutinis efektas  
0.8 – didelis efektas  

Todėl 0.568 rodo vidutinio dydžio efektą.

Tai reiškia, kad skirtumas tarp grupių yra ne tik statistiškai reikšmingas (pagal t testą), bet ir praktiškai reikšmingas, nors efektas nėra labai didelis.


## 2) Porinis t testas: „prieš“ ir „po“ palyginimas

Verslo klausimas:
- Ar po kainodaros pakeitimo klientų mėnesinės išlaidos padidėjo?

Duomenys:
- tie patys klientai matuojami du kartus (prieš ir po).
- ši situacija reikalauja porinio t testo, nes stebėjimai susieti.

Dažna klaida:
- poriniams duomenims taikomas nepriklausomų imčių t testas.


In [8]:
np.random.seed(7)

n_customers = 50
before = np.random.normal(loc=120, scale=25, size=n_customers)
# tarkime, po pokyčio vidutiniškai +8 EUR, bet su triukšmu
after = before + np.random.normal(loc=8, scale=15, size=n_customers)

t_stat_paired, p_value_paired = stats.ttest_rel(after, before)
t_stat_paired, p_value_paired


(4.870447205269689, 1.2046137681231163e-05)

Interpretacija:
- jei p ≤ α, galima teigti, kad „po“ reikšmės statistiškai skiriasi nuo „prieš“.
- praktinis vertinimas: vidutinis pokytis (after - before).

Atliktas porinis t testas (ttest_rel), kuriuo lyginami tie patys klientai prieš ir po pokyčio.

t statistika = 4.87  
p reikšmė = 0.000012

Teigiama t reikšmė rodo, kad „after“ vidurkis yra didesnis nei „before“.

p reikšmė yra gerokai mažesnė už 0.05, todėl nulinė hipotezė (kad pokyčio nėra) atmetama.

Išvada: vidutinės išlaidos po pokyčio statistiškai reikšmingai padidėjo. Skirtumas labai mažai tikėtinas atsitiktinis.



In [9]:
avg_change = (after - before).mean()
avg_change


9.762806805182038

## 3) Kategoriniai duomenys: chi kvadrato testas

Verslo klausimas:
- Ar mokėjimo metodas priklauso nuo kliento tipo (Member vs Normal)?

Duomenys:
- dvi kategorijos: CustomerType ir PaymentMethod.
- sudaroma kontingencijos lentelė ir taikomas chi kvadrato nepriklausomumo testas.

Dažna klaida:
- chi kvadrato testui naudojami procentai vietoje dažnių.


In [10]:
np.random.seed(10)

n = 400
customer_type = np.random.choice(["Member", "Normal"], size=n, p=[0.55, 0.45])

# sukuriama nedidelė priklausomybė: Members dažniau renkasi Ewallet
payment = []
for ct in customer_type:
    if ct == "Member":
        payment.append(np.random.choice(["Cash", "Card", "Ewallet"], p=[0.35, 0.30, 0.35]))
    else:
        payment.append(np.random.choice(["Cash", "Card", "Ewallet"], p=[0.45, 0.35, 0.20]))

df_pay = pd.DataFrame({"CustomerType": customer_type, "Payment": payment})

ct_table = pd.crosstab(df_pay["CustomerType"], df_pay["Payment"])
ct_table


Payment,Card,Cash,Ewallet
CustomerType,,,
Member,53,80,90
Normal,56,86,35


In [11]:
chi2, p, dof, expected = stats.chi2_contingency(ct_table)
chi2, p, dof


(19.466885842485844, 5.92678877094638e-05, 2)

Atliktas chi-kvadrato nepriklausomumo testas.

χ² statistika = 19.47  
p reikšmė = 0.000059  
laisvės laipsniai (dof) = 2  

p reikšmė yra daug mažesnė nei 0.05, todėl nulinė hipotezė atmetama. 

Tai reiškia, kad kliento tipas ir mokėjimo būdas nėra nepriklausomi – tarp jų egzistuoja statistiškai reikšmingas ryšys.

Praktiškai tai patvirtina, kad „Member“ klientai dažniau renkasi Ewallet, o „Normal“ klientų pasirinkimai pasiskirsto kitaip, ir šis skirtumas nėra atsitiktinis.


Interpretacija:
- p ≤ α rodo statistiškai reikšmingą ryšį tarp kintamųjų.
- papildomai verta pasižiūrėti, kurios ląstelės labiausiai prisideda prie skirtumo (observed vs expected).

Geroji praktika:
- įvertinti, ar laukiamų dažnių reikšmės nėra per mažos.


In [12]:
expected_df = pd.DataFrame(expected, index=ct_table.index, columns=ct_table.columns)
(ct_table - expected_df).round(1)


Payment,Card,Cash,Ewallet
CustomerType,,,
Member,-7.8,-12.5,20.3
Normal,7.8,12.5,-20.3


## 4) Proporcijų palyginimas (dažnas A/B atvejis): konversijos rodiklis

Verslo klausimas:
- Ar grupės B konversija (pirkimas) skiriasi nuo grupės A?

Tai yra proporcijų (dvejetainių rezultatų) palyginimas.
Dažnas pasirinkimas: z testas dviem proporcijoms (statsmodels).


In [14]:
# statsmodels gali nebūti reikalingas, tačiau proporcijoms jis yra patogus
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest


In [15]:
np.random.seed(21)

nA, nB = 1200, 1200
convA = np.random.binomial(1, 0.075, size=nA)  # 7.5% konversija
convB = np.random.binomial(1, 0.090, size=nB)  # 9.0% konversija

successes = np.array([convA.sum(), convB.sum()])
trials = np.array([nA, nB])

z_stat, p_val = proportions_ztest(count=successes, nobs=trials, alternative="two-sided")
successes, trials, z_stat, p_val


(array([89, 93]), array([1200, 1200]), -0.3084246973793868, 0.7577591915581285)

Grupė A: 89 konversijos iš 1200 → 7.42 %  
Grupė B: 93 konversijos iš 1200 → 7.75 %

z statistika = -0.31  
p reikšmė = 0.758  

Neigiama z reikšmė rodo, kad A konversijos rodiklis yra šiek tiek mažesnis nei B, tačiau skirtumas labai mažas.

p reikšmė (0.758) yra gerokai didesnė nei 0.05, todėl nulinė hipotezė neatmetama.

Išvada: statistiškai reikšmingo skirtumo tarp A ir B konversijų nenustatyta. Pastebėtas skirtumas greičiausiai yra atsitiktinis.


In [16]:
rateA = successes[0] / trials[0]
rateB = successes[1] / trials[1]

abs_pp = (rateB - rateA) * 100
rel_change = (rateB - rateA) / rateA * 100

rateA, rateB, abs_pp, rel_change


(0.07416666666666667, 0.0775, 0.3333333333333327, 4.494382022471901)

Grupė A konversijos rodiklis: 0.0742 → 7.42 %  
Grupė B konversijos rodiklis: 0.0775 → 7.75 %

Absoliutus skirtumas:
0.33 procentinio punkto (7.75 % − 7.42 %)

Santykinis pokytis:
+4.49 % lyginant su A

Tai reiškia, kad B grupėje konversija yra apie 4.5 % didesnė nei A, tačiau absoliutus skirtumas yra labai mažas – tik 0.33 procentinio punkto.

Kadangi ankstesnis z testas parodė, jog p = 0.758, šis skirtumas nėra statistiškai reikšmingas ir greičiausiai yra atsitiktinis.


## 5) Daugkartiniai palyginimai (poriniai palyginimai) ir klaidų lygis

Kai atliekami keli testai, didėja tikimybė bent kartą klaidingai atmesti H0 (pirmo tipo klaida).
Paprasta korekcija: Bonferroni.

Bonferroni idėja:
- jei atliekama m testų, kiekvienam testui taikomas α/m.

Kompromisas:
- sumažinama pirmo tipo klaidos rizika, bet padidėja antro tipo klaidos tikimybė.


In [17]:
def bonferroni_alpha(alpha, m):
    return alpha / m

bonferroni_alpha(0.05, 10)


0.005

## Dažnos klaidos ir geroji praktika

Dažnos klaidos:
1) Netinkamas testo pasirinkimas (poriniai duomenys testuojami kaip nepriklausomi).
2) Sprendimas priimamas vien pagal p reikšmę, ignoruojant efekto dydį ir verslo reikšmę.
3) Atliekama daug testų be korekcijos (išpučia pirmo tipo klaidos tikimybę).
4) Nepatikrinamos prielaidos ir duomenų kokybė (outlieriai, praleistos reikšmės, neteisingi filtrai).
5) Rezultatų „medžiojimas“ (p-hacking): bandymai su daug alternatyvių filtrų, kol p tampa mažas.

Geroji praktika:
1) Prieš testą aiškiai aprašyti H0, Ha, α ir metriką.
2) Įvertinti imties dydį ir praktinį efektą (kas laikoma reikšminga verslui).
3) Pateikti ne tik p reikšmę, bet ir:
   - vidurkių / proporcijų skirtumą,
   - efekto dydį,
   - konfidencinį intervalą (jei taikoma).
4) Naudoti Welch t testą, kai nėra užtikrinta apie vienodas dispersijas.
5) Dokumentuoti duomenų paruošimo žingsnius ir filtrus, kad analizė būtų atkartojama.


## Santrauka

- t testai: skaitinių rodiklių vidurkių palyginimui (nepriklausomų grupių ir poriniams duomenims).
- chi kvadrato testas: kategorinių kintamųjų ryšiui vertinti.
- proporcijų testas: konversijoms ir kitiems dvejetainiams rezultatams.
- daugkartiniai palyginimai: reikalinga klaidų kontrolė (pvz., Bonferroni).

Svarbiausia:
statistinis reikšmingumas nėra tas pats, kas verslo nauda. Sprendimą formuoja ir duomenys, ir kontekstas.
